In [3]:
# Start sqllite db
import sqlite3
import pandas as pd


In [6]:

conn = sqlite3.connect("clubs.db")
c = conn.cursor()


In [ ]:

# Make tables
c.execute(
    """CREATE TABLE clubs (
    club_id TEXT PRIMARY KEY,
    club_name TEXT,
    external_website TEXT,
    instagramUrl TEXT,
    facebookUrl TEXT,
    twitterUrl TEXT,
    twitterUserName TEXT,
    email TEXT,
    description TEXT,
    summary TEXT
)"""
)

c.execute(
    """CREATE TABLE IF NOT EXISTS categories (
    category_id INTEGER PRIMARY KEY,
    category_name TEXT
)"""
)


c.execute(
    """CREATE TABLE IF NOT EXISTS club_categories (
    club_id TEXT,
    category_id INTEGER,
    FOREIGN KEY (club_id) REFERENCES clubs(club_id),
    FOREIGN KEY (category_id) REFERENCES categories(category_id)
)"""
)


In [8]:
# Create media table
c.execute(
    """CREATE TABLE IF NOT EXISTS media (
    media_id INTEGER PRIMARY KEY,
    club_id TEXT,
    media_url TEXT,
    media_src TEXT,
    post_date TEXT,
    FOREIGN KEY (club_id) REFERENCES clubs(club_id)
    )
    """
)

# Add column club_name to media table
c.execute(
    """ALTER TABLE media
    ADD COLUMN club_name TEXT
    """
)

In [11]:

def table_summary(db_path, table_name, n=5):
    """
    Prints the first and last 'n' rows of the given table using Pandas for formatting.

    db_path: Path to the SQLite database file.
    table_name: Name of the table to print head and tail from.
    n: Number of rows from the head and tail to print. Default is 5.
    """
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)

        # Query to get the first 'n' rows (head)
        head_query = f"SELECT * FROM {table_name} LIMIT {n}"
        df_head = pd.read_sql_query(head_query, conn)

        # Query to get the last 'n' rows (tail)
        # In SQLite, we can't directly use OFFSET with a negative value, so we use a subquery
        tail_query = f"""
        SELECT * FROM (
            SELECT * FROM {table_name} ORDER BY ROWID DESC LIMIT {n}
        ) 
        """
        df_tail = pd.read_sql_query(tail_query, conn)

        # Print the head and tail using Pandas
        print(f"Head of {table_name}:")
        print(df_head.to_string(index=False))

        print("\n---\n")

        print(f"Tail of {table_name}:")
        print(df_tail.to_string(index=False))

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the connection
        conn.close()

In [12]:
table_summary('clubs.db', 'media')

Head of media:
Empty DataFrame
Columns: [media_id, club_id, media_url, media_src, post_date, club_name]
Index: []

---

Tail of media:
Empty DataFrame
Columns: [media_id, club_id, media_url, media_src, post_date, club_name]
Index: []


In [4]:
# Add insta_id column to clubs
conn = sqlite3.connect("clubs.db")
c = conn.cursor()
c.execute("ALTER TABLE clubs ADD COLUMN insta_id TEXT")
conn.close()